# Regressor setup demo

In this demo, we want to demonstartate how to use the different arguments in the model classes (LGT or DLT) to realize different setups for the regressors. Those could be very useful in practice when tuning the models.

In [2]:
import pandas as pd
import numpy as np
from orbit.lgt import LGT
from orbit.utils.plot import plot_predicted_data
from orbit.utils.plot import plot_predicted_components

## load data

In [3]:
DATA_FILE = "./data/iclaims.example.csv"

raw_df = pd.read_csv(DATA_FILE, parse_dates=['week'])

raw_df.dtypes

week              datetime64[ns]
claims                     int64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
dtype: object

In [4]:
raw_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


In [5]:
df=raw_df.copy()
test_size=52
train_df=df[:-test_size]
test_df=df[-test_size:]

## Use regressors and specify their signs

The regressor columns can be supplied via argument `regressor_col`. Their signs can be specified via `regressor_sign`, with values either '=' (regular, no restriction) or '+' (positive). These two lists should be of the same lenght. The default values of `regressor_sign` is all '='.

In [6]:
DATE_COL="week"
RESPONSE_COL="claims"
REGRESSOR_COL=['trend.unemploy', 'trend.filling', 'trend.job']

In [7]:
lgt_map=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='], 
    seasonality=52,
    seed=1,
    predict_method='map',
    is_multiplicative=True
)

In [8]:
lgt_map.fit(df=train_df)
_ = lgt_map.predict(df=test_df)

The estimated regressor coefficients can be retrieved via `.aggregated_posteriors`.

In [9]:
lgt_map.aggregated_posteriors['map']['pr_beta']

array([[4.15329225e-02, 1.12462424e-05]])

In [10]:
lgt_map.aggregated_posteriors['map']['rr_beta']

array([-0.07075831])

## Adjust pirors for regressor beta and regressor standard deviation

In the model, it is assumed $$\beta \sim Gaussian(\beta_{prior}, \sigma_{prior})$$

The default values for $\beta_{prior}$ and $\sigma_{prior}$ are 0 and 1, respectively.

Users could adjust them via arguments `regressor_beta_prior` and `regressor_sigma_prior`. These two lists should be of the same lenght as `regressor_col`.

In [11]:
lgt_map=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='], 
    regressor_beta_prior=[0.05, 0.05, 0],
    regressor_sigma_prior=[0.1, 0.1, 0.1],
    seasonality=52,
    seed=1,
    predict_method='map',
    is_multiplicative=True
)

In [12]:
lgt_map.fit(df=train_df)
_ = lgt_map.predict(df=test_df)

One can notice the significant changes in the estimted coefficients by using different priors.

In [13]:
lgt_map.aggregated_posteriors['map']['pr_beta']

array([[0.04529929, 0.06795404]])

In [14]:
lgt_map.aggregated_posteriors['map']['rr_beta']

array([-0.03710713])

## Use Cauchy prior on regressor standard deviation instead of fixed

Instead of using fixed standard deviations for regressors, a hyperprior can be assigned to them, i.e.
$$\sigma_\beta \sim \text{Half-Cauchy}(\sigma_{\beta_{prior}}, \sigma_{sd})$$

This can be done by setting `fix_regression_coef_sd = 0`, and $\sigma_{sd}$ can be changed via `regressor_sigma_sd`.

In [15]:
lgt_map=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='],
    fix_regression_coef_sd=0,
    regressor_sigma_sd=0.5,
    seasonality=52,
    seed=1,
    predict_method='map',
    is_multiplicative=True
)

In [16]:
lgt_map.fit(df=train_df)
_ = lgt_map.predict(df=test_df)

In [17]:
lgt_map.aggregated_posteriors['map']['pr_beta']

array([[4.14571092e-02, 1.00000000e-15]])

In [18]:
lgt_map.aggregated_posteriors['map']['rr_beta']

array([-0.07121057])

## Set max value threshold for regressor coef

In some cases, the users may want to tune the maximum threshold on the regressor coefficients. This can be done by using `regression_coef_max`, whose default value is 1.

In [19]:
lgt_map=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='],
    regression_coef_max=0.03,
    seasonality=52,
    seed=1,
    predict_method='map',
    is_multiplicative=True
)


lgt_map.fit(df=train_df)
_ = lgt_map.predict(df=test_df)

One can notice the impacts on the estimated coefficients.

In [20]:
lgt_map.aggregated_posteriors['map']['pr_beta']

array([[0.02995537, 0.02953037]])

In [21]:
lgt_map.aggregated_posteriors['map']['rr_beta']

array([-0.02876044])